original+augment

In [ ]:
# Import required libraries
from google.colab import drive
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split

# Step 1: Mount Google Drive
drive.mount('/content/drive')

# Step 2: Define the directory where your dataset is stored on Google Drive
data_dir = "/content/drive/MyDrive/original+augment"  # Replace with your dataset's path
batch_size = 32
num_classes = 6

# Step 3: Define transformations for the dataset
basic_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Step 4: Load the dataset from Google Drive
full_dataset = datasets.ImageFolder(root=data_dir, transform=basic_transforms)

# Step 5: Split the dataset into training and validation sets
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

# Step 6: Create data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Step 7: Load the EfficientNet B2 model and modify its classifier
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.efficientnet_b2(pretrained=True)
model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)
model = model.to(device)

# Step 8: Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Step 9: Define the training function
def train(model, loader, criterion, optimizer):
    model.train()
    running_loss = 0.0
    for inputs, labels in loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    return running_loss / len(loader.dataset)

# Step 10: Define the validation function
def validate(model, loader, criterion, num_classes):
    model.eval()
    running_loss = 0.0
    correct = 0

    tp = [0] * num_classes
    fp = [0] * num_classes
    fn = [0] * num_classes

    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)

            _, preds = torch.max(outputs, 1)
            correct += torch.sum(preds == labels.data)

            # Calculate TP, FP, FN for each class
            for i in range(num_classes):
                tp[i] += torch.sum((preds == i) & (labels == i)).item()
                fp[i] += torch.sum((preds == i) & (labels != i)).item()
                fn[i] += torch.sum((preds != i) & (labels == i)).item()

    accuracy = correct.double() / len(loader.dataset)
    avg_loss = running_loss / len(loader.dataset)

    for i in range(num_classes):
        print(f"Class {i} - TP: {tp[i]}, FP: {fp[i]}, FN: {fn[i]}")

    return avg_loss, accuracy, tp, fp, fn

# Step 11: Training loop
num_epochs = 7
for epoch in range(num_epochs):
    train_loss = train(model, train_loader, criterion, optimizer)
    val_loss, val_accuracy, tp, fp, fn = validate(model, val_loader, criterion, num_classes)
    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

# Step 12: Save the trained model to Google Drive
torch.save(model.state_dict(), "/content/drive/MyDrive/banana_disease_efficientnet_b2.pth")


Mounted at /content/drive


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B2_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_b2_rwightman-c35c1473.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b2_rwightman-c35c1473.pth
100%|██████████| 35.2M/35.2M [00:00<00:00, 130MB/s]


Class 0 - TP: 84, FP: 1, FN: 10
Class 1 - TP: 74, FP: 3, FN: 0
Class 2 - TP: 70, FP: 7, FN: 0
Class 3 - TP: 90, FP: 0, FN: 4
Class 4 - TP: 84, FP: 4, FN: 3
Class 5 - TP: 85, FP: 3, FN: 1
Epoch 1/7
Train Loss: 0.3547, Validation Loss: 0.1207, Validation Accuracy: 0.9644
Class 0 - TP: 87, FP: 0, FN: 7
Class 1 - TP: 70, FP: 0, FN: 4
Class 2 - TP: 70, FP: 3, FN: 0
Class 3 - TP: 93, FP: 3, FN: 1
Class 4 - TP: 87, FP: 3, FN: 0
Class 5 - TP: 86, FP: 3, FN: 0
Epoch 2/7
Train Loss: 0.1765, Validation Loss: 0.0707, Validation Accuracy: 0.9762
Class 0 - TP: 94, FP: 0, FN: 0
Class 1 - TP: 73, FP: 0, FN: 1
Class 2 - TP: 70, FP: 7, FN: 0
Class 3 - TP: 90, FP: 1, FN: 4
Class 4 - TP: 87, FP: 4, FN: 0
Class 5 - TP: 79, FP: 0, FN: 7
Epoch 3/7
Train Loss: 0.0849, Validation Loss: 0.0857, Validation Accuracy: 0.9762
Class 0 - TP: 94, FP: 1, FN: 0
Class 1 - TP: 74, FP: 0, FN: 0
Class 2 - TP: 69, FP: 0, FN: 1
Class 3 - TP: 94, FP: 0, FN: 0
Class 4 - TP: 87, FP: 0, FN: 0
Class 5 - TP: 86, FP: 0, FN: 0
Epoch 

original


In [ ]:
# Import necessary libraries
from google.colab import drive
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split

# Step 1: Mount Google Drive
drive.mount('/content/drive')

# Step 2: Define dataset directory in Google Drive
data_dir = "/content/drive/MyDrive/original"  # Adjust this to your dataset path
batch_size = 32
num_classes = 6

# Step 3: Define transformations for the dataset
basic_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Step 4: Load dataset and split into train/validation sets (80%/20%)
full_dataset = datasets.ImageFolder(root=data_dir, transform=basic_transforms)
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

# Step 5: Create data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Step 6: Load EfficientNet B2 and modify the classifier
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.efficientnet_b2(pretrained=True)
model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)
model = model.to(device)

# Step 7: Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Step 8: Define training function
def train(model, loader, criterion, optimizer):
    model.train()
    running_loss = 0.0
    for inputs, labels in loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    return running_loss / len(loader.dataset)

# Step 9: Define validation function
def validate(model, loader, criterion, num_classes):
    model.eval()
    running_loss = 0.0
    correct = 0

    tp = [0] * num_classes
    fp = [0] * num_classes
    fn = [0] * num_classes

    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)

            _, preds = torch.max(outputs, 1)
            correct += torch.sum(preds == labels.data)

            # Calculate TP, FP, FN for each class
            for i in range(num_classes):
                tp[i] += torch.sum((preds == i) & (labels == i)).item()
                fp[i] += torch.sum((preds == i) & (labels != i)).item()
                fn[i] += torch.sum((preds != i) & (labels == i)).item()

    accuracy = correct.double() / len(loader.dataset)
    avg_loss = running_loss / len(loader.dataset)

    for i in range(num_classes):
        print(f"Class {i} - TP: {tp[i]}, FP: {fp[i]}, FN: {fn[i]}")

    return avg_loss, accuracy, tp, fp, fn

# Step 10: Training loop
num_epochs = 7
for epoch in range(num_epochs):
    train_loss = train(model, train_loader, criterion, optimizer)
    val_loss, val_accuracy, tp, fp, fn = validate(model, val_loader, criterion, num_classes)
    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

# Step 11: Save the trained model to Google Drive
torch.save(model.state_dict(), "/content/drive/MyDrive/banana_disease_efficientnet_b2.pth")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Class 0 - TP: 11, FP: 4, FN: 2
Class 1 - TP: 32, FP: 5, FN: 0
Class 2 - TP: 8, FP: 3, FN: 5
Class 3 - TP: 17, FP: 0, FN: 0
Class 4 - TP: 5, FP: 0, FN: 0
Class 5 - TP: 1, FP: 0, FN: 5
Epoch 1/7
Train Loss: 0.9450, Validation Loss: 0.4394, Validation Accuracy: 0.8605
Class 0 - TP: 12, FP: 0, FN: 1
Class 1 - TP: 32, FP: 1, FN: 0
Class 2 - TP: 12, FP: 0, FN: 1
Class 3 - TP: 12, FP: 0, FN: 5
Class 4 - TP: 5, FP: 5, FN: 0
Class 5 - TP: 6, FP: 1, FN: 0
Epoch 2/7
Train Loss: 0.2304, Validation Loss: 0.1905, Validation Accuracy: 0.9186
Class 0 - TP: 11, FP: 0, FN: 2
Class 1 - TP: 32, FP: 0, FN: 0
Class 2 - TP: 13, FP: 0, FN: 0
Class 3 - TP: 17, FP: 0, FN: 0
Class 4 - TP: 5, FP: 1, FN: 0
Class 5 - TP: 6, FP: 1, FN: 0
Epoch 3/7
Train Loss: 0.1943, Validation Loss: 0.0876, Validation Accuracy: 0.9767
Class 0 - TP: 10, FP: 0, FN: 3
Class 1 - TP: 32, FP: 1, FN: 0
Class 2 -